In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.action_chains import ActionChains
import os.path
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys
def downloadChecker(download_path, file_prefix, first_phase):
        counter = 0
        while True:
            time.sleep(3)
            files = os.listdir(download_path)
            matching_files = [file for file in files if file.startswith(file_prefix)]
            if not matching_files and counter < 3:
                print(f" [*] {file_prefix} no exist. I'll check again.")
                counter += 1
                continue
            elif not matching_files and counter >= 3:
                if first_phase:
                    return False
                input(f" [*] {file_prefix} no exist.Press enter to recheck.")
                counter = 0
                continue
            else:
                print(f"[+] {file_prefix} downloaded successfully. Start reading.")
                if first_phase:
                    return True
                # Construct the full path to the first matching file
                file_to_check = os.path.join(download_path, matching_files[0])
                file_extension = os.path.splitext(file_to_check)[1]
                if os.path.isfile(file_to_check):
                    # Assuming it's an Excel file, you can read it into a Pandas DataFrame
                    try:
                        dictionary = pd.read_excel(file_to_check, sheet_name=None)
                        # Concatenate all DataFrames from each sheet into one DataFrame
                        df = pd.concat(dictionary, ignore_index=True)
                        print(f"[+] {file_prefix} read with excel successfully.")
                    except Exception as e:
                        if file_extension.lower() == '.csv':
                            try:
                                print(e)
                                df = pd.read_csv(file_to_check, encoding='utf-8')
                                print(f"[+] {file_prefix} read with csv successfully.")
                            except Exception as e:
                                print(e)
                        else:
                            try:
                                tables = pd.read_html(file_to_check)
                                df = tables[0]
                                print(f"[+] {file_prefix} read with html successfully.")
                            except Exception as e:
                                print(e)
                                input("Error occur while read the file.")
                    # You can now work with the 'df' DataFrame.
                    return df

In [8]:
out_of_stock_df = downloadChecker('/Users/papag/Downloads', "excel_downdload", first_phase=False)
# Read downloaded out of stock product list file
out_of_stock_df = downloadChecker('/Users/papag/Downloads', "excel_downdload", first_phase=False)
# Preprocessing
new_header = out_of_stock_df.iloc[0]  # Grab the first row for the header
out_of_stock_df = out_of_stock_df[1:]
out_of_stock_df.columns = new_header  # Set the header row as the df header
new_out_of_stock_df = out_of_stock_df.loc[out_of_stock_df['분류'].isin(['단종', '품절'])] # Grab the 상품코드 which 분류 is 단종 or 품절
new_out_of_stock_df.drop_duplicates(subset='상품코드', inplace=True)
new_out_of_stock_temp_df = out_of_stock_df.loc[out_of_stock_df['분류'].isin(['일시품절'])] # Grab the 상품코드 which 분류 is 단종 or 품절
new_out_of_stock_temp_df.drop_duplicates(subset='상품코드', inplace=True)
# Sending the out of stock prd codes to coupang and smartstre and delete them.
if len(new_out_of_stock_df) != 0:
    product_codes = new_out_of_stock_df['상품코드'].astype(str).tolist()
    out_of_stock_prd_string = ','.join(product_codes)
elif len(new_out_of_stock_df) == 0:
    out_of_stock_prd_string = "Empty"
if len(new_out_of_stock_temp_df) != 0:
    product_codes_temp = new_out_of_stock_temp_df['상품코드'].astype(str).tolist()
    out_of_stock_prd_temp_string = ','.join(product_codes_temp)
elif len(new_out_of_stock_temp_df) == 0:
    out_of_stock_prd_temp_string = "Empty"
print(out_of_stock_prd_string)
print(out_of_stock_prd_temp_string)

[+] excel_downdload downloaded successfully. Start reading.
[+] excel_downdload read with html successfully.
[+] excel_downdload downloaded successfully. Start reading.
[+] excel_downdload read with html successfully.
Empty
CH2281525


C:\Users\papag\AppData\Local\Temp\ipykernel_37592\3725024754.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_out_of_stock_df.drop_duplicates(subset='상품코드', inplace=True)


'CH2374883'

In [17]:
smart_df['PC\n기본 할인 단위'] = '%'
smart_df['모바일\n기본할인 단위'] = '%'
smart_df['']
smart_df = smart_df[:4]
smart_df

,상품번호,판매자 상품코드,카테고리코드,상품명,상품상태,판매가,부가세,재고수량,옵션형태,옵션명,...,알림받기 동의 \n고객 전용 여부,ISBN,ISSN,독립출판,출간일,출판사,글작가,그림작가,번역자명,문화비 소득공제
0,9828627670,CH2245707,50000780,커밍 코코 겨울 삼선 방한화 (핑크) 크록스내피 대용,신상품,13000,과세상품,100,조합형,옵션,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9828627565,CH2292541,50000780,탈부착퍼내피 크록스내피 대용,신상품,4900,과세상품,500,조합형,옵션,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9828056601,CH2347029,50000434,고슴도치 가방고리 키링 인형 [미니 귀여운 털 동물 도치 애착 봉제 키홀더 백참 반...,신상품,7200,과세상품,100,조합형,옵션,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9828056449,CH2347968,50000434,갸우뚱 오리 인형 키링 2color [귀여운 동물 캐릭터 가방고리 키링인형 선물] ...,신상품,14400,과세상품,200,조합형,옵션,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Define the values for each column
prd_codes = ['CH2292541', 'CH2245707', 'CH2347968', 'CH2347029']
discount_rates = [1, 2, 3, 4]

# Create the DataFrame
smart_discount_rate_df = pd.DataFrame({
    'prd_code': prd_codes,
    'discount_rate': discount_rates
})

In [19]:
# Step 1: Create a mapping from prd_code to discount_rate
discount_map = smart_discount_rate_df.set_index('prd_code')['discount_rate'].to_dict()

# Step 2: Update the discount values in smart_df
smart_df['PC\n기본 할인 값'] = smart_df['판매자 상품코드'].map(discount_map)
smart_df['모바일\n기본할인 값'] = smart_df['판매자 상품코드'].map(discount_map)
smart_df['PC\n기본 할인 값']

0    2
1    1
2    4
3    3
Name: PC\n기본 할인 값, dtype: int64

There are number of 50 items.
